In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from rag_utils.vanilla import vectorstore

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

In [2]:
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever()

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.vanilla import format_docs, rag_prompt, llm

vanilla_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.sub_query import SubQueryRetriever

sub_query_retriever = SubQueryRetriever.from_vectorstore(vectorstore)

sub_query_chain = (
    {"context": sub_query_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [6]:
query = "What is the difference between MRKL and HuggingGPT?"

vanilla_result = vanilla_rag_chain.invoke(query)
sub_query_result = sub_query_chain.invoke(query)
print(
    f"\n[vanilla_result]:\n{vanilla_result}\n\n[sub_query_result]:\n{sub_query_result}"
)

sub_queries: ['What is MRKL?', 'What is HuggingGPT?']

[vanilla_result]:
I'm sorry, but the provided context does not contain any information about MRKL, so I cannot provide a comparison between MRKL and HuggingGPT.

[sub_query_result]:
MRKL (Modular Reasoning, Knowledge and Language) and HuggingGPT are both systems that utilize large language models (LLMs), but they differ in their structure and functionality. 

MRKL, proposed by Karpas et al. in 2022, is a neuro-symbolic architecture for autonomous agents. It contains a collection of "expert" modules, and the LLM works as a router to route inquiries to the most suitable expert module. These modules can be neural, like deep learning models, or symbolic, like a math calculator or weather API.

On the other hand, HuggingGPT, proposed by Shen et al. in 2023, is a framework that uses ChatGPT as a task planner to select models available in the HuggingFace platform according to the model descriptions. It then summarizes the response based o